In [3]:
import pickle
import dota2api
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None

#Open match ID list and item dictionary to process data
pickle_in = open("match_ID_list.pickle","rb")
match_ID_list = pickle.load(pickle_in)

pickle_in = open("item_dictionary.pickle","rb")
item_dictionary = pickle.load(pickle_in)

#My DotA API Web Key 
WebKey = #Your personal key goes here
api = dota2api.Initialise(WebKey)

#Define player ID
player_id = #Your player ID goes here

#Create match table: each match ID is a keyword in a dictionary, and match details are a mixed-value list of items
match_table = {}

#Function to help find whether player was on the winning team or not.
def find_player_index(list_arg, key, value):
    for i, dic in enumerate(list_arg):
        if dic[key] == player_id:
            return i

for ID in match_ID_list:
    
    match = api.get_match_details(match_id = ID)
    
    match_details_for_specified_player = []
    match_details_for_specified_player.append(match['match_id'])
    match_details_for_specified_player.append(match['start_time'])
    match_details_for_specified_player.append(match['lobby_type'])
    match_details_for_specified_player.append(match['lobby_name'])
    match_details_for_specified_player.append(match['game_mode'])
    match_details_for_specified_player.append(match['duration'])

    #find the detailed information about given player
    for player in match['players']:

        if player.get('account_id', -1) == player_id:
            match_details_for_specified_player.append(player['kills'])
            match_details_for_specified_player.append(player['deaths'])
            match_details_for_specified_player.append(player['assists'])
            
            match_details_for_specified_player.append(player['last_hits'])
            match_details_for_specified_player.append(player['level'])

            match_details_for_specified_player.append(player['xp_per_min'])
            match_details_for_specified_player.append(player['gold_per_min'])

            match_details_for_specified_player.append(player.get('gold_spent', -1))
            match_details_for_specified_player.append(player.get('gold', -1))

            match_details_for_specified_player.append(player['hero_name'])
            
            #Use the dictionary to convert them into strings
            match_details_for_specified_player.append(item_dictionary[player['item_0']])
            match_details_for_specified_player.append(item_dictionary[player['item_1']])
            match_details_for_specified_player.append(item_dictionary[player['item_2']])
            match_details_for_specified_player.append(item_dictionary[player['item_3']])
            match_details_for_specified_player.append(item_dictionary[player['item_4']])
            match_details_for_specified_player.append(item_dictionary[player['item_5']])
            
            match_details_for_specified_player.append(item_dictionary[player['backpack_0']])
            match_details_for_specified_player.append(item_dictionary[player['backpack_1']])
            match_details_for_specified_player.append(item_dictionary[player['backpack_2']])

    #Leaver detector: in case no stats are recorded, returns -1
    leaver_status = 0 
    for player in match['players']:
        if player.get('leaver_status', -1) > 1:
            leaver_status = 1
            break
        elif player.get('leaver_status', -1) == -1:
            leaver_status = -1
            break
    match_details_for_specified_player.append(leaver_status)
    
    #Find if the player won or lost the match
    try:
        if find_player_index(match['players'], 'account_id', player_id) <= 4 and match['radiant_win'] == True:
            match_details_for_specified_player.append(1)
        elif find_player_index(match['players'], 'account_id', player_id) > 4 and match['radiant_win'] == False:
            match_details_for_specified_player.append(1)
        else:
            match_details_for_specified_player.append(0)
    except KeyError: 
        match_details_for_specified_player.append(-1)
        
    match_table[ID] = match_details_for_specified_player
    
#create dataframe from the dictionary
match_dataframe = pd.DataFrame.from_dict(match_table, orient='index')

#reset the index of dictionary keys with default indexing
match_dataframe = match_dataframe.reset_index(drop=True)

#assign column names based on the order they were recorded in the dictionary value lists
match_dataframe.columns = ['MatchID', 'Date', 'LobbyType', 'LobbyName', 'GameMode', 'Duration', 
                           'Kills', 'Deaths', 'Assists', 'LastHits', 'Level', 'ExpPerMin', 
                           'GoldPerMin', 'GoldSpent', 'GoldRemaining', 'Hero', 
                           'Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 
                           'Backpack1', 'Backpack2', 'Backpack3', 
                           'LeaverStatus', 'Won']

#convert timestamps into standard date format
match_dataframe['Date'] = pd.to_datetime(match_dataframe.Date, unit = 's')
#delete LobbyName column, since it is simply a string repeat of LobbyType column
del match_dataframe['LobbyName']

#convert duration from seconds to HH:MM:SS format
match_dataframe['Duration'] = pd.to_timedelta(match_dataframe['Duration'], unit='s')

#Store the result            
pickle_out = open("match_dataframe.pickle","wb")
pickle.dump(match_dataframe, pickle_out)
pickle_out.close()

#Check if it saved
pickle_in = open("match_dataframe.pickle","rb")
check_match_dataframe = pickle.load(pickle_in)

display(check_match_dataframe)